In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

/home/edvard/.local/share/virtualenvs/Mastering_PyTorch-ib8cv033/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(799)
tkz = GPT2Tokenizer.from_pretrained("gpt2")
mdl = GPT2LMHeadModel.from_pretrained('gpt2')
ln = 10
cue = "They"
gen = tkz(cue, return_tensors="pt")
to_ret = gen["input_ids"][0]

/home/edvard/.local/share/virtualenvs/Mastering_PyTorch-ib8cv033/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
prv=None
for i in range(ln):
    outputs = mdl(**gen)
    next_token_logits = torch.argmax(outputs.logits[-1, :])
    to_ret = torch.cat([to_ret, next_token_logits.unsqueeze(0)])
    gen = {"input_ids": to_ret}

seq = tkz.decode(to_ret)

print(seq)

They are not the only ones who are being targeted.


In [4]:
ip_ids = tkz.encode(cue, return_tensors='pt')
op_greedy = mdl.generate(ip_ids, max_length=ln, pad_token_id=tkz.eos_token_id)
seq = tkz.decode(op_greedy[0], skip_special_tokens=True)
print(seq)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


They are not the only ones who are being targeted


In [5]:
op_beam = mdl.generate(
    ip_ids, 
    max_length=5, 
    num_beams=3, 
    num_return_sequences=3,
    pad_token_id=tkz.eos_token_id
)

for op_beam_cur in op_beam:
    print(tkz.decode(op_beam_cur, skip_special_tokens=True))

They have a lot of
They have a lot to
They are not the only


In [6]:
for i in range(3):
    torch.manual_seed(i+10)
    op = mdl.generate(
        ip_ids, 
        do_sample=True, 
        max_length=5, 
        top_k=2,
        pad_token_id=tkz.eos_token_id
    )

    seq = tkz.decode(op[0], skip_special_tokens=True)
    print(seq)

They are the only ones
They have been in the
They have a lot of


In [7]:
for i in range(3):
    torch.manual_seed(i+10)
    op = mdl.generate(
        ip_ids, 
        do_sample=True, 
        max_length=5, 
        top_p=0.75, 
        top_k=0,
        pad_token_id=tkz.eos_token_id
    )

    seq = tkz.decode(op[0], skip_special_tokens=True)
    print(seq)

They in the jail know
They live on a budget
They

Says
